In [ ]:
# ONLY USE THESE IMPORTS.
# PLEASE DON'T EDIT THIS CELL.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read Data (Edit)

In [ ]:
x_train_frog = np.array([plt.imread('Data/train/frog/' + str(i) + '.jpg').reshape(-1) for i in range(0, 5000)])
x_train_frog_temp = x_train_frog
x_train_airplane = np.array([plt.imread('Data/train/airplane/' + str(i) + '.jpg').reshape(-1) for i in range(0, 5000)])
x_train_airplane_temp = x_train_airplane
x_train_automobile = np.array(
    [plt.imread('Data/train/automobile/' + str(i) + '.jpg').reshape(-1) for i in range(0, 5000)])

x_train_automobile_temp = x_train_automobile
x_test_frog = np.array([plt.imread('Data/test/frog/' + str(i) + '.jpg').reshape(-1) for i in range(0, 1000)])
x_test_frog_temp = x_test_frog
x_test_airplane = np.array([plt.imread('Data/Test/airplane/' + str(i) + '.jpg').reshape(-1) for i in range(0, 1000)])
x_test_airplane_temp = x_test_airplane
x_test_automobile = np.array(
    [plt.imread('Data/test/automobile/' + str(i) + '.jpg').reshape(-1) for i in range(0, 1000)])
x_test_automobile_temp = x_test_automobile



<hr style="border:2px solid black"> </hr>

## Construct Fischer's Linear Discriminant classifier for each of the 3 classes.
#### Test each classifier on all images in X_Test. 
#### Construct Confusion Matrix.

In [ ]:
def normalize(array):
    array = np.divide(array, 255)
    return array


x_train_frog = normalize(x_train_frog)
x_train_automobile = normalize(x_train_automobile)
x_train_airplane = normalize(x_train_airplane)
x_test_frog = normalize(x_test_frog)
x_test_automobile = normalize(x_test_automobile)
x_test_airplane = normalize(x_test_airplane)

In [ ]:
def reshape(x_train_frog, x_train_automobile, x_train_airplane):
    x_train = x_train_airplane
    x_train = np.concatenate((x_train, x_train_frog))
    x_train = np.concatenate((x_train, x_train_automobile))
    return (x_train)


x_train = reshape(x_train_frog, x_train_automobile, x_train_airplane)


In [ ]:
def classifier(array, value1, value2, value3):
    mean = []
    mean_not_a = []
    x = []
    for i in range(0, value3):
        a = array[value1:value2, i]
        mean.append(np.mean(a))
    for i in range(0, value3):
        x.append(np.subtract(array[value1:value2, i], mean[i]))
    transpose = np.transpose(np.matrix(x))
    sw_a = np.matmul(x, transpose)
    if value1 == 0 and value2 == 5000:
        x = []
        for i in range(0, value3):
            a = array[value2:15000, i]
            mean_not_a.append(np.mean(a))
        for i in range(0, value3):
            x.append(np.subtract(array[value2:15000, i], mean_not_a[i]))
        transpose = np.transpose(np.matrix(x))
        sw_not_a = np.matmul(x, transpose)
    elif value1 == 5000 and value2 == 10000:
        x = []
        array = np.concatenate((array[0:5000, :], array[10000:15000, :]))
        for i in range(0, value3):
            a = array[:, i]
            mean_not_a.append(np.mean(a))
        for i in range(0, value3):
            x.append(np.subtract(array[:, i], mean_not_a[i]))
        transpose = np.transpose(np.matrix(x))
        sw_not_a = np.matmul(x, transpose)
    else:
        x = []
        for i in range(0, value3):
            a = array[0:10000, i]
            mean_not_a.append(np.mean(a))
        for i in range(0, value3):
            x.append(np.subtract(array[0:10000, i], mean_not_a[i]))
        transpose = np.transpose(np.matrix(x))
        sw_not_a = np.matmul(x, transpose)

    sw = np.add(sw_a, sw_not_a)
    sw_1 = np.linalg.pinv(sw)
    mean_difference = np.subtract(mean, mean_not_a)
    mean_difference = mean_difference[:, None]
    classify = np.matmul(sw_1, mean_difference)
    mean_addition = np.add(mean, mean_not_a)
    mean_addition_m = np.multiply(mean_addition, (-0.5))
    mean_addition_m = mean_addition_m[:, None]
    scalar = np.matmul(np.transpose(classify), mean_addition_m)
    return scalar[0][0], classify


In [ ]:
airplane_w0, airplane_classifier = classifier(x_train, 0, 5000, 3072)
frog_w0, frog_classifier = classifier(x_train, 5000, 10000, 3072)
automobile_w0, automobile_classifier = classifier(x_train, 10000, 15000, 3072)


In [ ]:
def predict(x_test_auto, x_test_air, x_test_f, w0, w1, w2, airclass, autoclass, frogclass):
    y_test_airplane = []
    y_test_automobile = []
    y_test_frog = []
    countairf = 0
    countair = 0
    countairauto = 0
    cm = []
    for i in range(len(x_test_air)):
        y_test_airplane.append(np.dot(x_test_air[i], airclass) + w0)
        y_test_automobile.append(np.dot(x_test_auto[i], airclass) + w0)
        y_test_frog.append(np.dot(x_test_f[i], airclass) + w0)
        predicted = max(y_test_airplane[i], y_test_automobile[i], y_test_frog[i])
        if (predicted[0] == y_test_airplane[i]):
            countair += 1
        elif (predicted[0] == y_test_automobile[i]):
            countairauto += 1
        else:
            countairf += 1
    countfrog = 0
    countfrogauto = 0
    countfrogair = 0
    print(countair, countairf, countairauto)
    y_test_airplane = []
    y_test_automobile = []
    y_test_frog = []
    for i in range(len(x_test_f)):
        y_test_airplane.append(np.dot(x_test_air[i], frogclass) + w1)
        y_test_automobile.append(np.dot(x_test_auto[i], frogclass) + w1)
        y_test_frog.append(np.dot(x_test_f[i], frogclass) + w1)
        predicted = max(y_test_airplane[i], y_test_automobile[i], y_test_frog[i])
        if predicted[0] == y_test_airplane[i]:
            countfrogair += 1
        elif predicted[0] == y_test_automobile[i]:
            countfrogauto += 1
        else:
            countfrog += 1
    print([countfrog, countfrogauto, countfrogair])
    countauto = 0
    countautof = 0
    countautoair = 0
    y_test_airplane = []
    y_test_automobile = []
    y_test_frog = []
    for i in range(len(x_test_auto)):
        y_test_airplane.append(np.dot(x_test_air[i], autoclass) + w2)
        y_test_automobile.append(np.dot(x_test_auto[i], autoclass) + w2)
        y_test_frog.append(np.dot(x_test_f[i], autoclass) + w2)
        predicted = max(y_test_airplane[i], y_test_automobile[i], y_test_frog[i])
        if (predicted[0] == y_test_airplane[i]):
            countautoair += 1
        elif (predicted[0] == y_test_automobile[i]):
            countauto += 1
        else:
            countautof += 1
    print(countauto, countautof, countautoair)
    cm = [[countair, countairf, countairauto], [countfrogair, countfrog, countfrogauto],
          [countautoair, countautof, countauto]]
    airpredict = max(countair, countairf, countairauto)
    frogpredict = max(countfrog, countfrogauto, countfrogair)
    autopredict = max(countauto, countautoair, countautof)
    return cm

## Confusion Matrix (Don't Edit)

In [ ]:
cm = predict(x_test_automobile, x_test_airplane, x_test_frog, airplane_w0, frog_w0, automobile_w0, airplane_classifier,
             automobile_classifier, frog_classifier)

# Test


confusion_matrix = np.array(cm)

accuracy = np.diag(confusion_matrix).sum() / 3000
plt.rc('figure', figsize=[5, 5])
plt.matshow(confusion_matrix, cmap="Blues")
plt.title('FLD+RGB = {0:0.3f}'.format(accuracy))
for i in range(0, confusion_matrix.shape[0]):
    for j in range(0, confusion_matrix.shape[1]):
        plt.annotate(confusion_matrix[i, j], (j, i))
plt.show()

### Repeat for Grayscale

In [ ]:
def togray(image):
    count = 0
    im = []
    while count + 3 <= len(image[0]):
        r, g, b = image[:, count], image[:, count + 1], image[:, count + 2]
        r = np.multiply(r, 0.2989)
        g = np.multiply(g, 0.5870)
        b = np.multiply(b, 0.1140)
        np.add(r, g, b)
        im.append(np.add(r, g, b))
        count = count + 3
    im = np.transpose(im)
    return im

In [ ]:
x_train_frog_grayscale = normalize(x_train_frog_temp)
x_train_frog_grayscale = togray(x_train_frog_grayscale)

x_train_automobile_grayscale = normalize(x_train_automobile_temp)
x_train_automobile_grayscale = togray(x_train_automobile_grayscale)

x_train_airplane_grayscale = normalize(x_train_airplane_temp)
x_train_airplane_grayscale = togray(x_train_airplane_grayscale)

x_train_grayscale = reshape(x_train_frog_grayscale, x_train_automobile_grayscale, x_train_airplane_grayscale)

airplanew0grey, airplane_grayscale = classifier(x_train_grayscale, 0, 5000, 1024)
frogw0grery, frog_grayscale = classifier(x_train_grayscale, 5000, 10000, 1024)
autow0grey, automobile_grayscale = classifier(x_train_grayscale, 10000, 15000, 1024)


x_test_auto_grayscale = normalize(x_test_automobile_temp)
x_test_auto_grayscale = togray(x_test_auto_grayscale)
x_test_airplane_grayscale = normalize(x_test_airplane_temp)
x_test_airplane_grayscale = togray(x_test_airplane_grayscale)
x_test_frog_grayscale = normalize(x_test_frog_temp)
x_test_frog_grayscale = togray(x_test_frog_grayscale)

## Confusion Matrix (Don't Edit)

In [ ]:
cm = predict(x_test_auto_grayscale, x_test_airplane_grayscale, x_test_frog_grayscale, airplanew0grey, frogw0grery,
             autow0grey, airplane_grayscale, automobile_grayscale, frog_grayscale)

confusion_matrix = np.array(cm)


accuracy = np.diag(confusion_matrix).sum() / 3000
plt.rc('figure', figsize=[5, 5])
plt.matshow(confusion_matrix, cmap="Blues")
plt.title('FLD+GRAY = {0:0.3f}'.format(accuracy))
for i in range(0, confusion_matrix.shape[0]):
    for j in range(0, confusion_matrix.shape[1]):
        plt.annotate(confusion_matrix[i, j], (j, i))
plt.show()

<hr style="border:2px solid black"> </hr>

## Comment on the change of accuracy between RGB and Grayscale
##########


The comments are in the report
uploaded (ml.docx)
We uploaded two other files (main.py) and this colab file , main.py is our original python file as we worked on pycharm as  the upload of files would take long time with colab and we copied the .py file in this .ipnyb file we uploaded both 
thank you,
...
...

##########
<hr style="border:2px solid black"> </hr>